In [34]:
from sympy import symbols, Eq, solve
import numpy as np

#нужно эти значения подставить в метод ньютона -----
"""
Physical Equations
"""
def dV(Q):
    return Q


def dQ(V, Q, R=10, I=0.0003, C=10):
    return -Q * R / I - V / I / C


def system_i(t, var, i, Res, Ext):
    V, Q = var
    R, I, C = Res[i], 0.1, Ext[i]
    
    return np.array([dV(Q), dQ(V, Q, R, I, C)])


def full_system(t, var, params=avg_normal_params):
    Res, Sq, Ext = params
    res = []
    for i in range(0, len(var), 2):
        v = var[i:i+2]
        res.append(system_i(t, v, i // 2, Res, Ext))
        
    return np.array(res).flatten()




def RK_system(t0, t1, y_init, 
              system = full_system, 
              params = avg_normal_params,
              n = 1000):
    """
    y0 = V(t0) = V0, Q(t0) = Q0
    """
    
    tz = t0
    y0 = y_init.copy()
    h = (t1 - tz) / n
    
    for i in range(n):
        k1 = system(tz, y0, params)
        k2 = system(tz + h/2, y0 + k1 * h/2, params)
        k3 = system(tz + h/2, y0 + k2 * h/2, params)
        k4 = system(tz + h, y0 + k3 * h, params)
        k = h * (k1 + 2*k2 + 2*k3 + k4) / 6
        
        tz += h
        y0 += k
        
    return np.round(y0, 4)


def error(left, right, 
          params=avg_normal_params, 
          ro=1.05):
    """
    1. Mass conservation
    2. Momentum conservation
    """
    Res, Sq, Ext = params
    left = left.reshape((len(Graph), -1))
    right = right.reshape((len(Graph), -1))
    err = []
        
    for i, ves in enumerate(Graph):
        V_i = right[i][0]
        Q_i = right[i][1] 
        moment_i = Q_i**2 / 2 / Sq[i]**2 + V_i / Ext[i] / ro
        
        for to in ves:
            V_to = left[to][0]
            Q_to = left[to][1] 
        
            # mass
            V_i -= V_to
            
            # momentum
            moment_i -= Q_to**2 / 2 / Sq[to]**2 + V_to / Ext[to] / ro
        
        err += [V_i, moment_i]
    
    return np.array(err)




In [35]:
def shoot(t0, t1, 
          y_init = np.random.uniform(1, 5, len(Graph)*2),
          system = full_system,
          params = avg_normal_params,
          solver = RK_system,
          error = error,
          delta = 10**(-4),
          debug = False):

    t = (t1 - t0) / 2
    yt = y_init.copy()
    
    eps = 10**(-4)
    
    cnt = 0
    while True:
        
        left = solver(t, t0, yt, system, params)
        right = solver(t, t1, yt, system, params)
        err = error(left, right, params)
        
        # Converged
        if np.all(abs(err) < delta):
            cnt += 1
            print(f'Converged in {cnt} iterations', end='')
            break
        # find Jacobian and its determinant
        J = np.empty((len(err), len(yt)))
        for i in range(len(yt)):
            yi = yt.copy()
            yi[i] += eps
            
            left = solver(t, t0, yi, system, params)
            right = solver(t, t1, yi, system, params)
            res = error(left, right, params)
            
            J[:, i] = (res - err) / eps
        
        D = det(J)
        
        # Critical point
        if abs(D) < 10**(-10):
            cnt += 1
            print(f'Critical point found in {cnt} iterations | Error = {np.round(err, 3)}')
            break
        
        # Update yt and calculate new error 
        yt -= np.dot(np.linalg.inv(J),  err)
        
    
        # Iteration limit exceeded
        cnt += 1
        if cnt == 30:
            return np.nan

        if debug:
            print(cnt, D)
            
    D = round(D, 4)

    if debug:
        return cnt, yt, D
    
    return D        

In [ ]:
avg_normal_params = np.array([[0.002, 0.002],
                              [350, 160], 
                              [0.32, 0.22]]) 

In [36]:
#уравнения до обезразмеривания
from scipy.optimize import fsolve
import math

params = np.array([[2, 3],
                   [260, 300],
                   [0ю3, 2.5]])
T=0.3


I = 0.0003
R1, R2, S1, S2, C1, C2 = params.flatten()
A1 = R1/I
B1 = 1/I/C1
A2 = R2/I
B2 = 1/I/C2
a1, b1, a2, b2 = 0, 8, 8, 21

alp11 = (np.sqrt(A1**2 - 4*B1) + A1) * (b1 - a1) / 2
alp12 = (np.sqrt(A1**2 - 4*B1) - A1) * (b1 - a1) / 2
alp21 = (np.sqrt(A2**2 - 4*B2) + A2) * (b2 - a2) / 2
alp22 = (np.sqrt(A2**2 - 4*B2) - A2) * (b2 - a2) / 2



def equations_a(p):
    K11,K12, K21, K22 = p
    
    v1_a1 = K11*np.exp(-a1*alp11) + K12*np.exp(a1*alp12)
    v1_b1 = K11*np.exp(-b1*alp11) + K12*np.exp(b1*alp12)
    
    v2_a2 = K21*np.exp(-a2*alp21) + K22*np.exp(a2*alp22)
    v2_b2 = K21*np.exp(-b2*alp21) + K22*np.exp(b2*alp22)
    
    dv1_a1 = -alp11*K11*np.exp(-a1*alp11) + alp12*K12*np.exp(a1*alp12)
    dv1_b1 = -alp11*K11*np.exp(-b1*alp11) + alp12*K12*np.exp(b1*alp12)
    
    dv2_a2 = -alp21*K21*np.exp(-a2*alp21) + alp22*K22*np.exp(a2*alp22)
    dv2_b2 = -alp21*K21*np.exp(-b2*alp21) + alp22*K22*np.exp(b2*alp22)
    
    eq1 = v1_a1 - v2_b2
    eq2 = v1_b1 - v2_a2
    eq3 = dv1_a1**2/S1**2/2 + v1_a1/C1/1.05 - dv2_b2**2/S2**2/2 - v2_b2/C2/1.05
    eq4 = dv1_b1**2/S1**2/2 + v1_b1/C1/1.05 - dv2_a2**2/S2**2/2 - v2_a2/C2/1.05 
    
    return (eq1, eq2, eq3, eq4)

K11,K12, K21, K22 = fsolve(equations_a, (10, 10, 10, 10))
K11,K12, K21, K22

(-4.16688065410481e-15,
 4.166881021373002e-15,
 -216270174.96122098,
 1.0221732922487467e-13)

In [37]:
v1_a1 = K11*np.exp(-a1*alp11) + K12*np.exp(a1*alp12)
v1_b1 = K11*np.exp(-b1*alp11) + K12*np.exp(b1*alp12)

v2_a2 = K21*np.exp(-a2*alp21) + K22*np.exp(a2*alp22)
v2_b2 = K21*np.exp(-b2*alp21) + K22*np.exp(b2*alp22)
#решения
v1_a1, v1_b1, v2_a2, v2_b2

(3.672681919735443e-22,
 9.710035300071511e-20,
 9.710183011147143e-20,
 1.5885270148081406e-29)

In [ ]:
#нужно эти значения подставить в метод ньютона -----

In [18]:
#уравнения после обезразмеривания

from scipy.optimize import fsolve
import math

params = np.array([[0.2, 0.3],
                   [2.60, 3.00],
                   [0.3, 0.25]])
T=0.3
I = 0.0003
R1, R2, S1, S2, C1, C2 = params.flatten()
A1 = R1/I
B1 = 1/I/C1
A2 = R2/I
B2 = 1/I/C2
a1, b1, a2, b2 = 0, 8, 8, 21
alp11 = (np.sqrt(A1**2 - 4*B1) + A1) 
alp12 = (np.sqrt(A1**2 - 4*B1) - A1) 
alp21 = (np.sqrt(A2**2 - 4*B2) + A2) 
alp22 = (np.sqrt(A2**2 - 4*B2) - A2) 
al11=np.sqrt(A1**2 - 4*B1)
al22=np.sqrt(A2**2 - 4*B2)


def equations_a(p):
    K11,K12, K21, K22 = p
    v1_a1 = K11*np.exp((-0.5*a1)*alp11) + K12*np.exp(0.5*a1*alp12)
    v1_b1 = K11*np.exp((-0.5*b1)*alp11) + K12*np.exp(0.5*b1*alp12)  
    v2_a2 = K21*np.exp((-0.5*a2)*alp21) + K22*np.exp(0.5*a2*alp22)
    v2_b2 = K21*np.exp((-0.5*b2)*alp21) + K22*np.exp(0.5*b2*alp22) 
    dv1_a1 = 0.5*np.exp((-0.5*a1)*alp11)*(((K12*np.exp(a1*al11))*al11)-(A1*(K12*np.exp(a1*al11))))
    dv1_b1 = 0.5*np.exp((-0.5*b1)*alp11)*(((K12*np.exp(b1*al11))*al11)-(A1*(K12*np.exp(b1*al11))))
    dv2_a2 =0.5*np.exp((-0.5*a2)*alp21)*(((K22*np.exp(a2*al22))*al22)-(A2*(K22*np.exp(a2*al22)))) 
    dv2_b2 = 0.5*np.exp((-0.5*b2)*alp21)*(((K22*np.exp(b2*al22))*al22)-(A2*(K22*np.exp(b2*al22)))) 

    
    V1ch=3.0
    V2ch=3.5
    ro=1.05
    eq1 = ((V1ch/T)*v1_a1*(0/T)) -((V2ch/T)*v2_b2*(2.1/T))
    eq2 = ((V1ch/T)*v1_b1*(0/T)) - ((V2ch/T)*v2_a2*(2.1/T))
    eq3 = (((V1ch/T)*((dv1_a1*(8/T))/(1.414*S1)))**2)+((v1_a1*(8/T))/(ro*C1))-(((V2ch/T)*((dv2_b2*(8/T))/(1.414*S2)))**2)-((v2_b2*(8/T))/(ro*C2))

    eq4 = (((V1ch/T)*((dv1_b1*(8/T))/(1.414*S1)))**2)+((v1_b1*(8/T))/(ro*C1))-(((V2ch/T)*((dv2_a2*(8/T))/(1.414*S2)))**2)-((v2_a2*(8/T))/(ro*C2))
    
    return (eq1, eq2, eq3, eq4)

K11,K12, K21, K22 = fsolve(equations_a, (10, 10, 10, 10))
K11,K12, K21, K22

<ipython-input-18-860324d359dd>:32: RuntimeWarning: overflow encountered in exp
  dv1_b1 = 0.5*np.exp((-0.5*b1)*alp11)*(((K12*np.exp(b1*al11))*al11)-(A1*(K12*np.exp(b1*al11))))
<ipython-input-18-860324d359dd>:32: RuntimeWarning: invalid value encountered in double_scalars
  dv1_b1 = 0.5*np.exp((-0.5*b1)*alp11)*(((K12*np.exp(b1*al11))*al11)-(A1*(K12*np.exp(b1*al11))))
<ipython-input-18-860324d359dd>:33: RuntimeWarning: overflow encountered in exp
  dv2_a2 =0.5*np.exp((-0.5*a2)*alp21)*(((K22*np.exp(a2*al22))*al22)-(A2*(K22*np.exp(a2*al22))))
<ipython-input-18-860324d359dd>:33: RuntimeWarning: invalid value encountered in double_scalars
  dv2_a2 =0.5*np.exp((-0.5*a2)*alp21)*(((K22*np.exp(a2*al22))*al22)-(A2*(K22*np.exp(a2*al22))))
<ipython-input-18-860324d359dd>:34: RuntimeWarning: overflow encountered in exp
  dv2_b2 = 0.5*np.exp((-0.5*b2)*alp21)*(((K22*np.exp(b2*al22))*al22)-(A2*(K22*np.exp(b2*al22))))
<ipython-input-18-860324d359dd>:34: RuntimeWarning: invalid value encountered in doub

(10.0, 10.0, 10.0, 10.0)

In [19]:
v1_a1 = K11*np.exp(-a1*alp11) + K12*np.exp(a1*alp12)
v1_b1 = K11*np.exp(-b1*alp11) + K12*np.exp(b1*alp12)

v2_a2 = K21*np.exp(-a2*alp21) + K22*np.exp(a2*alp22)
v2_b2 = K21*np.exp(-b2*alp21) + K22*np.exp(b2*alp22)

v1_a1, v1_b1, v2_a2, v2_b2

(20.0, 1.3751310042040862e-118, 1.2053167340037613e-93, 2.903291305928971e-246)

In [32]:
#вот что то посчиталось даже адекватное 
Graph = [[1], [2]]
N_vessels = len(Graph)

RK_system(0, 0.8, y_init = np.random.uniform(1, 5, len(Graph)*2), 
              system = full_system, 
              params = avg_normal_params,
              n = 1000)


array([-1.6996, 17.9965,  1.0092, 16.2639])

In [33]:
from scipy import optimize
a = np.arange(-50, 50)
root = optimize.newton(RK_system, 1.5, args = (a,avg_normal_params ))

ValueError: not enough values to unpack (expected 2, got 1)

In [30]:
shoot(0, 1, 
          y_init = np.random.uniform(1, 5, len(Graph)*2),
          system = full_system,
          params = avg_normal_params,
          solver = RK_system,
          error = error,
          delta = 10**(-4),
          debug = False)

NameError: name 'shoot' is not defined